In [10]:
import requests
import datetime
import pandas as pd

# Sends a Get Request and convert Response to JSON
def requestRespone( url ):
    responseFromServer = requests.get(url)
    responseJson = responseFromServer.json()
    return responseJson;

# create data frame from google sheet to have extensibility 
df = pd.read_csv('https://docs.google.com/spreadsheet/ccc?key=1dVz0Og5sZvedwEaQZ_ULjv3gYc-wntjT69NDpY49k4U&output=csv')
# Finds the minimum date in data 
minimumDateInDate = df['date_first'].min()
# Finds the maximum date in data 
maximumDateInData = df['date_last'].max()

# In order to get google sheet data in the form of JSON
base_url = 'https://script.google.com/macros/s/AKfycbygukdW3tt8sCPcFDlkMnMuNu9bH5fpt7bKV50p2bM/exec?id=1dVz0Og5sZvedwEaQZ_ULjv3gYc-wntjT69NDpY49k4U&sheet=locations.csv'
responseInJson = requestRespone(base_url)

# initialization of lists and Variables
locationId = []
precipProbability = []
days = []
noOfDays = 0
firstIteration = 0

# For each location, on each day gets precipitation Probability
for location in responseInJson['locations.csv']:
    # In order to get coordinates from Postal code
    url = 'http://api.postcodes.io/postcodes/'
    postalCode = location.get('postal_code')
    # Get minimum date for that location
    dateFirst = location.get('date_first')
    # Get maximum date for that location
    dateLast = location.get('date_last')
    # Adding postal code to the url
    url = url + postalCode
    currentStartDate = minimumDateInDate
    locationId.append(location.get('loc_id'))
    # Gets coordinates from api.postcodes.io API
    cordinatesResponse = requestRespone(url)
    while currentStartDate <= maximumDateInData:
        # Adding all dates in the dates List 
        if firstIteration is 0:
            date = datetime.datetime.fromtimestamp(int(currentStartDate)).strftime('%Y-%m-%d')
            days.append(date)
        #  If the date is not in the given range or the postal code is not correct then add 'NaN' in the List
        if (currentStartDate < dateFirst) or (currentStartDate > dateLast) or (cordinatesResponse['status'] == 404):
            precipProbability.append('NaN')
        else:
            latitude = cordinatesResponse['result'].get('latitude')
            longitude = cordinatesResponse['result'].get('longitude')
            url = 'https://api.darksky.net/forecast/8b29191b53f16224e5d45f9d91455762/'
            # Creating URL for getting precipitation Probability from darksky API
            url = url + str(latitude) + ", " + str(longitude) + ", " + str(currentStartDate) + '?exclude=currently,flags'
            daysPrecipProbabilityResponse = requestRespone(url)
            prepResponse = daysPrecipProbabilityResponse['daily'].get('data')
            # if prepResponse[0] (i.e. daily data) does not contain precipProbability then add 'NaN' 
            # in the precipProbability list 
            if "precipProbability" not in prepResponse[0]:
                precipProbability.append('NaN')
            else:
                precipProbability.append(str(prepResponse[0]['precipProbability']) + '%')
        # adding 86400 to go to next day, since each day has 86400 seconds
        currentStartDate = currentStartDate + 86400
        noOfDays = noOfDays + 1
    firstIteration = 1

numberOfDaysInLargestRange = int(noOfDays/locationId.__len__())
# Convert list of precipitation Probability into list of lists where records in each sub list are
# precipitation Probabilities of a location id in the same row
precipProbabilityForEachDay = [precipProbability[x:x+numberOfDaysInLargestRange] for x in range(0, len(precipProbability), numberOfDaysInLargestRange)]

df = pd.DataFrame(precipProbabilityForEachDay, columns = days, index =  locationId )
df.index.name = 'Location ID'
df.columns.name = 'Days'
df


Days,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-07,2017-02-08,2017-02-09,...,2017-02-19,2017-02-20,2017-02-21,2017-02-22,2017-02-23,2017-02-24,2017-02-25,2017-02-26,2017-02-27,2017-02-28
Location ID,,,,,,,,,,,,,,,,,,,,,
9be000ae23275d57e1273d211a54ffd7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a35e427b4130be7b2a892e286f0ebb91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0%,0%,0.67%,0.66%,0%,0.52%,0.64%,0.57%,0.74%,0.61%
185674a2eb5c14fbdbb1d05a4109ea55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
afbddd7f957a1c822293616e95a2d84c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81cea1e224ad183b751acce139f4e276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5f1ce9b7c8cd32c08d98310540fb6604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0%,0.98%,0%,0.97%,0%,0.98%,0.97%,0.93%,0.98%,0.98%
5180af03094779de849ca816c9f5b753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7f2aa8e72612f9130e06b32a0d2a58d7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8b049b660e984912c48da213f2f7c650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
